# LOAD CARRYING FEATURE

## 1. INITIATION

### 1.1 API

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from scipy import signal
from pyts.image import GramianAngularField
from keras.utils import to_categorical

### 1.2 Font

In [ ]:
LabelFont = {'family' : 'Times New Roman',
             'weight' : 'normal',
             'size'   : 25}

UnitFont = {'family' : 'Times New Roman',
            'weight' : 'normal',
            'size'   : 20}

TextFont = {'family' : 'Times New Roman',
            'weight' : 'normal',
            'size'   : 35}

### 1.3 Supporting File

In [ ]:
%run "2.IMU_Set.ipynb"
%run "3.Feature.ipynb"
%run "3.Data_Feature.ipynb"
%run "5.Filter_Lowpass.ipynb"

## 2. FEATURE

### 2.1 Initial Value

In [ ]:
# Lopass Filter # 
Order = 5
Cutoff = 10
DelayNumber = 100
SR = 128

In [ ]:
# Load-carrying Range #
Range = {
    
    'S1' : [
             [860,1740],[2420,3280],[4250,5060],[5800,6730],[7600,8470],[9250,10080],
             [10950,11700],[12500,13350],[14230,15020],[15810,16630],[17490,18250],[19030,19920],

             [47950,48770],[49450,50290],[50980,51810],[52360,53310],[53970,54770],[55560,56390],
             [56960,57740],[58760,59680],[60580,61450],[62100,62940],[63640,64570],[65390,66280],

             [92870,93620],[94570,95360],[96800,97670],[98640,99630],[100400,101400],[102300,103200],
             [104000,104900],[105800,106600],[107600,108600],[109300,110200],[111000,111900],[112800,113600],

             [191100,191900],[192600,193400],[194300,195300],[195800,196700],[197300,198200],[198900,199800],
             [200500,201500],[202200,203300],[204200,205000],[205900,206800],[207500,208400],[209300,210300]
           ],

    'S2' : [
             [2535,3686],[5160,6230],[7556,8629],[9848,10780],[11880,12900],[14010,14950],
             [16410,17470],[18430,19630],[20550,21670],[22690,23890],[24790,25850],[26660,27730],

             [45000,46090],[46850,47910],[48730,49860],[50870,51910],[52690,53720],[54610,55770],
             [56790,57900],[58680,59820],[60640,61760],[62660,63790],[64640,65760],[66590,67660],

             [84380,85220],[86000,87020],[87770,88850],[89770,90860],[91750,92700],[93530,94560],
             [95520,96440],[97280,98330],[99220,100200],[101000,102200],[103000,103900],[104800,106000],

             [123300,124300],[125000,126000],[126600,127800],[128700,129700],[130500,131500],[132300,133300],
             [134100,135200],[136000,137000],[137900,138900],[139700,140800],[141500,142600],[143300,144400]
           ],
    
    'S3' : [
        
             [9090,10020],[11790,12610],[14500,15350],[16260,17200],[18410,19200],[20150,21020],
             [21770,22660],[23810,24830],[26170,27040],[27620,28670],[29810,30640],[31500,32270],
        
             [55020,56090],[56650,57520],[58470,59430],[60050,61120],[62270,63160],[64360,65260],
             [65930,66900],[67590,68440],[69540,70410],[71160,72140],[73020,73800],[74680,75580],
        
             [112700,113600],[114400,115300],[116500,117500],[118400,119400],[120300,121200],[122000,123000],
             [123700,124800],[125500,126400],[127600,128600],[129200,130200],[131000,132000],[132800,133700],
        
             [159700,160700],[161400,162300],[163100,164000],[164700,165800],[166900,167700],[168800,169700],
             [170300,171400],[172200,173200],[174400,175400],[176300,177400],[178400,179300],[180300,181300]
           ],
    
    'S4' : [
        
             [2010,3095],[4155,5395],[6300,7500],[8395,9690],[10940,12140],[13090,14220],
             [15060,16240],[17170,18490],[19320,20390],[21260,22610],[23560,24680],[25460,26530],
        
             [42030,43090],[44050,45430],[46200,47340],[48050,49340],[50490,51630],[52540,53670],
             [54650,55780],[56860,57960],[58900,60030],[60960,62300],[63220,64360],[65320,66310],
        
             [86430,87630],[88650,89970],[90860,92080],[93090,94280],[95510,96440],[97360,98380],
             [99580,100700],[102300,103400],[104300,105400],[106400,107500],[108800,110000],[110900,112000],
        
             [129900,131000],[132500,133800],[134600,135800],[136900,138000],[139200,140400],[141500,142500],
             [143500,144800],[146100,147200],[148300,149400],[150200,151400],[152500,153700],[154600,155600]        
           ],
    
    'S5' : [
             [633,1538],[2427,3391],[4370,5307],[6291,7221],[8038,9233],[9788,10710],
             [11590,12480],[13290,14240],[15080,16090],[16990,17910],[18780,19700],[20500,21430],

             [66540,67430],[68140,69140],[69940,70850],[71700,72590],[73400,74310],[75170,76070],
             [77060,77950],[78700,79620],[80530,81420],[82530,83420],[84260,85190],[86020,86940],
        
             [158300,159200],[160100,161000],[161700,162700],[163700,164600],[165400,166400],[167200,168100],
             [169000,169900],[170800,171800],[172600,173500],[174400,175300],[176100,177100],[177900,178900],
        
             [243200,244100],[244800,245700],[246500,247400],[248400,249400],[250200,251100],[251900,252900],
             [253900,254800],[255600,256600],[257300,258200],[259100,260000],[260800,261700],[262500,263400]        
           ],
    
    'S6' : [
             [6117,7180],[8128,9247],[10290,11540],[12620,13720],[14690,15810],[16700,17970],
             [19220,20380],[21310,22390],[23220,24330],[25680,26630],[27680,28620],[29550,30520],
        
             [61810,62750],[63800,64910],[65970,67100],[68360,69470],[70390,71390],[72500,73630],
             [74880,76000],[77070,78040],[79220,80310],[81440,82500],[83470,84600],[85700,86840],
        
             [105900,106800],[107800,108700],[109800,110800],[112100,113000],[114000,115000],[116100,117200],
             [118300,119300],[120400,121400],[122500,123500],[124700,125700],[126800,127700],[128700,129900],
        
             [145900,147000],[147900,149100],[149900,151000],[152000,153100],[154100,155200],[156200,157300],
             [158600,159600],[160600,161600],[162700,163900],[164900,166000],[167100,168000],[169100,170100]        
           ],
    
    'S8' : [
        
             [17260,18130],[19010,19930],[20920,21840],[22830,23860],[24750,25760],[26480,27400],
             [28310,29250],[30300,31330],[31960,32760],[33670,34600],[35900,36780],[37410,38450],
        
             [51240,52160],[53270,54310],[55500,56420],[57040,57930],[58930,59930],[60660,61560],
             [62610,63490],[64450,65420],[66030,66950],[67920,68930],[69920,70790],[71490,72390],
     
             [81870,82810],[84090,85040],[86140,87100],[88010,88950],[90040,90980],[91900,92840],
             [93780,94720],[95830,96780],[97620,98570],[99560,100500],[102000,103000],[103600,104600],
        
             [120400,121400],[122600,123700],[124400,125300],[126000,127000],[128300,129300],[130200,131100],
             [131900,132900],[134000,134900],[135600,136500],[137400,138300],[139400,140300],[141000,142000]
           ],
    
    'S9' : [
             [1132,1947],[3406,4287],[5816,6604],[7486,8286],[9557,10460],[11220,12060],
             [13230,14000],[15470,16260],[17030,17940],[18670,19580],[20450,21430],[22140,22930],
        
             [44100,44900],[45590,46410],[46950,47780],[48460,49270],[50040,50820],[51280,52120],
             [52750,53650],[54240,55030],[55560,56340],[56930,57810],[58760,59600],[60220,61010],
        
             [84080,84990],[85540,86510],[87010,87980],[88390,89390],[90050,90910],[91390,92360],
             [92840,93840],[94440,95290],[96190,97180],[97810,98780],[99420,100400],[100800,101800],
        
             [127900,128900],[129200,130300],[131200,132100],[132600,133600],[134100,135000],[135800,136700],
             [137300,138300],[138700,139700],[140300,141100],[141600,142500],[143100,143900],[144600,145500]        
           ],
    
    'S10' : [
             [500,1586],[2443,3531],[4224,5309],[6783,7896],[8831,10100],[10870,11990],
             [12980,14220],[14970,16070],[17080,18220],[19310,20420],[21420,22490],[23340,24400],
        
             [55920,56980],[57750,58860],[59690,60790],[61870,62980],[63840,64960],[66290,67560],
             [68500,69620],[70460,71560],[72520,73640],[74700,75860],[76620,77710],[78580,79650],
        
             [98310,99500],[100200,101500],[102300,103500],[104500,105600],[106300,107600],[108300,109500],
             [110500,111600],[112300,113400],[114400,115600],[116500,117700],[118600,119800],[120500,121700],
        
             [140200,141400],[142300,143200],[144200,145300],[146200,147400],[148100,149300],[150100,151300],
             [152100,153300],[154100,155100],[155800,157000],[158000,159200],[159800,161100],[161700,162700]
            ],
    
    'S11' : [
             [378,1469],[2443,3588],[4200,5385],[6443,7573],[8430,9541],[10340,11500],
             [12580,13730],[14680,15850],[16540,17600],[18460,19560],[20530,21660],[22350,23400],
        
             [51660,52770],[53750,54730],[55420,56590],[57330,58470],[59350,60470],[61090,62330],
             [63240,64380],[65200,66360],[66990,68070],[68880,70040],[70980,72030],[72560,73600],
        
             [111100,112300],[113000,114200],[114900,116100],[117100,118100],[119200,120200],[120900,122000],
             [122900,124100],[125000,126100],[126900,127900],[128900,130000],[130900,132000],[132700,133800],
        
             [164200,165300],[166100,167200],[167800,168900],[169700,170700],[171800,172800],[173800,174900],
             [175900,176900],[177900,179000],[179600,180600],[181500,182500],[183400,184400],[185200,186200]
            ],
    
    'S12' : [
             [4880,5953],[6429,7538],[8619,9770],[10460,11440],[11990,13080],[13710,14830],
             [15530,16680],[17080,18250],[18870,19890],[20550,21720],[22160,23120],[23850,24830],
        
             [51960,52850],[53350,54320],[54930,55890],[56740,57550],[58120,59230],[59820,60810],
             [61570,62530],[63220,64220],[64940,66090],[66750,67720],[68420,69390],[70040,70990],
        
        
             [99620,100600],[101200,102200],[102800,103900],[104400,105500],[106200,107200],[107900,108800],
             [109500,110600],[111200,112200],[112900,113800],[114600,115500],[116300,117300],[117900,119000],
        
             [157100,158000],[158700,159600],[160300,161200],[162000,162900],[163600,164600],[165300,166200],
             [166900,167900],[168500,169400],[170100,171000],[171700,172600],[173300,174200],[174800,175800]
            ],
    
    'S13' : [
             [479,1570],[2726,3870],[4997,6104],[7132,8083],[9347,10400],[11120,12060],
             [13040,14120],[15040,15980],[16840,17800],[18580,19750],[20620,21560],[22220,23150],
        
             [41640,42550],[43510,44460],[45160,46070],[47040,47970],[49110,50060],[50810,51720],
             [52600,53550],[54460,55420],[56130,56960],[57940,58870],[59920,60880],[61580,62540],

        
             [83580,84410],[86230,87180],[87830,88770],[89890,90850],[92070,93050],[93790,94750],
             [95850,96690],[98000,98950],[99680,100700],[101600,102400],[103600,104500],[105300,106300],
        
             [124600,125500],[126600,127600],[128200,129200],[130000,131100],[132300,133300],[134000,135000],
             [135900,137000],[137900,139100],[139800,140800],[141500,142600],[143700,144700],[145500,146500]
            ],
    
    'S14' : [
             [400,1300],[2737,3637],[4562,5462],[6417,7317],[8158,9058],[9780,10680],
             [11600,12500],[13190,14090],[14690,15590],[16360,17260],[17870,18770],[19310,20210],
        
             [44070,44970],[45480,46380],[47090,47990],[48700,49600],[50120,51020],[51650,52550],
             [53480,54380],[54940,55840],[56500,57400],[58150,59050],[59590,60490],[61190,62090],
        
        
             [98720,99620],[100300,101200],[101800,102700],[103500,104400],[104900,105800],[106300,107200],
             [108000,108900],[109300,110200],[110900,111800],[112600,113500],[114000,114900],[115700,116600],

             [147900,148800],[149300,150200],[150900,151800],[152400,153300],[153800,154700],[155200,156100],
             [157000,157900],[158300,159200],[159700,160600],[161300,162200],[162800,163700],[164400,165300]
            ],
    
    'S15' : [
             [1336,2236],[3289,4189],[4931,5831],[6635,7535],[8405,9305],[10090,10990],
             [11750,12650],[13430,14330],[15040,15940],[16570,17470],[18300,19200],[19890,20790],

             [44420,45320],[45990,46890],[47550,48450],[49170,50070],[50980,51880],[52520,53420],
             [54330,55230],[56150,57050],[57820,58720],[59600,60500],[61400,62300],[63000,63900],
        
        
             [89150,90050],[91010,91910],[92690,93590],[94540,95440],[96320,97220],[97970,98870],
             [99730,100630],[101400,102300],[103000,112000],[104700,105600],[106400,107300],[108000,108900],

             [133200,134100],[134700,135600],[136400,137300],[138100,139000],[139800,140700],[141400,142300],
             [143200,144100],[144900,145800],[146500,147400],[148100,149000],[149900,150800],[151500,152400]  
            ]
    
        }

In [ ]:
# Load-carrying Weigth Label #
Label_Weight_Total = {
    
    'S1' : [0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,
            0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2],
    'S2' : [2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,
            1,0,2,1,0,2,1,0,2,1,0,2,2,0,1,2,0,1,2,0,1,2,0,1],
    'S3' : [0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,
            0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1],
    'S4' : [1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,
            1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0],
    'S5' : [2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,
            2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1],
    'S6' : [2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,
            2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1],
    'S8' : [1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,
            1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0],
    'S9' : [2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,
            2,1,0,2,1,0,2,1,0,2,1,0,0,1,2,0,1,2,0,1,2,0,1,2],
   'S10' : [2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,
            2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1],
   'S11' : [2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,
            2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0],
   'S12' : [1,0,2,1,0,2,1,0,2,1,0,2,2,0,1,2,0,1,2,0,1,2,0,1,
            2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1],
   'S13' : [2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,
            2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0],
   'S14' : [2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,
            2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1,2,0,1],
   'S15' : [2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,
            2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0,2,1,0]

                     }

In [ ]:
# Load-carrying posture Label #
Label_Posture_Total = {
    
    'S1' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S2' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S3' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S4' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S5' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S6' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S8' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S9' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
            1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S10' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S11' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
             1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S12' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
             0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S13' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
             1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    'S14' : [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
             0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'S15' : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
             1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

                       }

### 2.2 File Name

In [ ]:
DIR_Data = '../1. Data/'
DIR_Subject = ['S1','S2','S3','S4','S5','S6','S8','S9','S10','S11','S12','S13','S14','S15']
DIR_IMU = ['Acc_L', 'Acc_R', 'Acc_W', 'Ang_L', 'Ang_R', 'Ang_W']

### 2.3 Feature Extraction

In [ ]:
for Subject in DIR_Subject:
    
    Range_Sub = Range[Subject]
    Label_Weight_Sub = Label_Weight_Total[Subject]
    Label_Posture_Sub = Label_Posture_Total[Subject]
    
    IMU_Set_Sub_Temp = IMU_Set()
    
    for i in range(0,6):
        
        IMU_Set_Sub = []

        for j in range(0,3):

            IMU_Set_Sub.append(IMU_Set_Sub_Temp[i*3+j])
            
        Feature_Set, Label_Weight_Set, Label_Posture_Set = Data_Feature(Range_Sub, Label_Weight_Sub, Label_Posture_Sub)
        
        Label_Weight_Set = to_categorical(Label_Weight_Set, num_classes = 3)
        Label_Posture_Set = to_categorical(Label_Posture_Set, num_classes = 2)
        

        # GAF #
        Feature_Set = Data_GAF(Feature_Set)

        Feature_Name = Subject + '_' + DIR_IMU[i] + '_Feature.csv'
        Label_Weight_Name = Subject + '_' + DIR_IMU[i] + '_Label_Weight.csv'
        Label_Posture_Name = Subject + '_' + DIR_IMU[i] + '_Label_Posture.csv'

        Feature_Save()